In [ ]:
pip install osmnx

In [ ]:
pip install geopandas

In [ ]:
pip install ipyleaflet

In [ ]:
import networkx as nx
import osmnx as ox
from shapely.geometry import LineString, mapping
import geopandas as gpd
from ipyleaflet import *

In [ ]:
place_name = 'Los Olivos, Lima, Peru'
graph = ox.graph_from_place(place_name, network_type='drive')
graph_edit=nx.DiGraph(graph)

In [ ]:
nodes, edges = ox.graph_to_gdfs(graph)
edges

In [ ]:
edges = edges[edges['name']!='Avenida Holanda']
graph_edit=ox.graph_from_gdfs(nodes,edges)

In [ ]:
center = (-11.9548679, -77.0797236)
m = Map(center=center, basemap=basemaps.OpenStreetMap.France, zoom=15)

In [ ]:
to_marker= Icon(
    icon_url='https://th.bing.com/th/id/R.fa74fd8669452ada18cfae1d2cdfe09e?rik=TrnRqmfWVA6PXg&pid=ImgRaw&r=0',
)
from_marker = Marker(location=center)
to_marker = Marker(location=center, icon=to_marker)

In [ ]:
def set_nearest_node(marker):
 marker.nearest_node = ox.get_nearest_node(graph, marker.location)
 return

In [ ]:
path_layer_list = []
def handle_change_location(event, marker):
    event_owner = event['owner']
    event_owner.nearest_node = ox.get_nearest_node(graph, event_owner.location)
    marker.nearest_node = ox.get_nearest_node(graph, marker.location)
    
    shortest_path = nx.dijkstra_path(graph_edit, event_owner.nearest_node, marker.nearest_node, weight='length')                          
    
    if len(path_layer_list) == 1:
        m.remove_layer(path_layer_list[0])
        path_layer_list.pop()
    
    shortest_path_points = nodes.loc[shortest_path]
    path = gpd.GeoDataFrame([LineString(shortest_path_points.geometry.values)], columns=['geometry'])
    path_layer = GeoData(geo_dataframe=path, style={'color':'green', 'weight':4})
    m.add_layer(path_layer)
    path_layer_list.append(path_layer)

In [ ]:
from_marker.observe(lambda event: handle_change_location(event, to_marker), 'location')
to_marker.observe(lambda event: handle_change_location(event, from_marker), 'location')

In [ ]:
m.add_layer(from_marker)
m.add_layer(to_marker)
set_nearest_node(from_marker)
set_nearest_node(to_marker)
m